In [0]:
%sql
CREATE DATABASE IF NOT EXISTS training;

In [0]:
%sql
CREATE OR REPLACE TABLE training.EMP_SCD1
(
EMPNO INT, 
	ENAME STRING, 
	JOB STRING, 
	MGR INT, 
	HIREDATE DATE, 
	SAL INT, 
	COMM INT, 
	DEPTNO INT
)
USING DELTA
LOCATION '/FileStore/tables/delta-table-merge/EMP';

In [0]:
%sql
select * from training.EMP_SCD1

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO


In [0]:
SOURCE_PATH = "dbfs:/FileStore/tables/EMP1-4.csv"
df_source = spark.read.options(header=True, delimiter=',', inferSchema='True').csv(SOURCE_PATH)
#spark.read.options(header=True, delimiter=',',inferSchema='True')\
df_source.printSchema()
display(df_source)


root
 |-- EMPNO: integer (nullable = true)
 |-- ENAME: string (nullable = true)
 |-- JOB: string (nullable = true)
 |-- MGR: integer (nullable = true)
 |-- HIREDATE: string (nullable = true)
 |-- SAL: integer (nullable = true)
 |-- COMM: integer (nullable = true)
 |-- DEPTNO: integer (nullable = true)



EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,17-Nov-81,5000,50,10
7698,BLAKE,MANAGER,7839,1-May-81,2850,2000,30
7782,CLARK,MANAGER,7839,9-Jun-81,2450,10,10
7566,JONES,MANAGER,7839,2-Apr-81,2975,50,20
7788,SCOTT,ANALYST,7566,19-Apr-87,3000,20,20
7902,FORD,ANALYST,7566,3-Dec-81,3000,50,20
7369,SMITH,CLERK,7902,17-Dec-80,800,500,20
7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30
7521,WARD,SALESMAN,7698,22-Feb-81,1250,500,30
7654,MARTIN,SALESMAN,7698,28-Sep-81,1250,1400,30


In [0]:
df_source.createOrReplaceTempView('EMP_SOURCE')

In [0]:
%sql
select * from EMP_SOURCE

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,17-Nov-81,5000,null,10
7698,BLAKE,MANAGER,7839,1-May-81,2850,null,30
7782,CLARK,MANAGER,7839,9-Jun-81,2450,null,10
7566,JONES,MANAGER,7839,2-Apr-81,2975,null,20
7788,SCOTT,ANALYST,7566,19-Apr-87,3000,null,20
7902,FORD,ANALYST,7566,3-Dec-81,3000,null,20
7369,SMITH,CLERK,7902,17-Dec-80,800,null,20
7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30
7521,WARD,SALESMAN,7698,22-Feb-81,1250,500,30
7654,MARTIN,SALESMAN,7698,28-Sep-81,1250,1400,30


In [0]:
%sql
/*
MERGE INTO training.EMP_SCD1 TARGET
USING EMP_SOURCE SOURCE
ON TARGET.EMPNO = SOURCE.EMPNO
WHEN MATCHED THEN
  UPDATE SET
    TARGET.ENAME = SOURCE.ENAME,
    TARGET.JOB = SOURCE.JOB,
    TARGET.MGR = SOURCE.MGR,
    TARGET.HIREDATE = SOURCE.HIREDATE,
    TARGET.SAL = SOURCE.SAL,
    TARGET.COMM = SOURCE.COMM,
    TARGET.DEPTNO = SOURCE.DEPTNO
WHEN NOT MATCHED THEN
  INSERT
  (
EMPNO,
ENAME,
JOB,
MGR,
HIREDATE,
SAL,
COMM,
DEPTNO
  )
  VALUES
  (
SOURCE.EMPNO,
SOURCE.ENAME,
SOURCE.JOB,
SOURCE.MGR,
SOURCE.HIREDATE,
SOURCE.SAL,
SOURCE.COMM,
SOURCE.DEPTNO
  )
*/

MERGE INTO training.EMP_SCD1 TARGET
USING EMP_SOURCE SOURCE
ON TARGET.EMPNO = SOURCE.EMPNO
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
16,16,0,0


In [0]:
%sql
select * from training.EMP_SCD1

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,null,5000,null,10
7698,BLAKE,MANAGER,7839,null,2850,null,30
7782,CLARK,MANAGER,7839,null,2450,null,10
7566,JONES,MANAGER,7839,null,2975,null,20
7788,SCOTT,ANALYST,7566,null,3000,null,20
7902,FORD,ANALYST,7566,null,3000,null,20
7369,SMITH,CLERK,7902,null,800,null,20
7499,ALLEN,SALESMAN,7698,null,1600,300,30
7521,WARD,SALESMAN,7698,null,1250,500,30
7654,MARTIN,SALESMAN,7698,null,1250,1400,30


In [0]:
# The below steps are used to perform SCD TYPE1 using PySpark.
#When the “Merge” Operation needs to be “Performed” using the “PySpark”, it is “Mandatory” to “Keep” the “Source” as the “DataFrame”. So, “No” “View” needs to be “Created” as the “Source”.
#The “Entire Data” of the “Target Delta Table” needs to be “Read” as the “Delta Table” first, in order to use the “Target” in the “Merge” Operation.

In [0]:
"""What is a “Delta Table Instance”?
A “Delta Table Instance” is a “Replica” of the “Actual Delta Table”.
The “Delta Table Instance” will “Create” a “Soft Link” to the “Actual Delta Table”, but “Not” a “Copy” of the “Actual Delta Table”.
Since, a “Soft Link” “Exists” between the “Delta Table Instance” and the “Actual Delta Table”, any “DML Operations” “Performed” on the “Delta Table Instance” will “Reflect” in the “Actual Delta Table”.
Similarly, any “Modification” “Performed” on the “Actual Delta Table” will “Reflect” in the “Delta Table Instance”.
To “Perform” any “DML Operation”, or, to “Execute” any “SQL Statement” on a “Delta Table” using “PySpark”, just “Referring” the “Delta Table” by its “Name” is “Not” going to “Work”.
To “Perform” any “DML Operation”, or, to “Execute” any “SQL Statement” on a “Delta Table” using “PySpark”, the “Delta Table Instance” of that “Delta Table” needs to be “Created”. This is the “Primary Usage” of the “Delta Table Instance”.
"""
from delta.tables import *

EMPDeltaTable = DeltaTable.forPath(spark, "/FileStore/tables/delta-table-merge/EMP")
print(type(EMPDeltaTable))

<class 'delta.tables.DeltaTable'>


In [0]:
"""In order to “Display” the “Data” of a “Delta Table Instance”, the “Delta Table Instance” needs to be “Converted” to a “DataFrame”.
To “Convert” a “Delta Table Instance” to a “DataFrame”, the “.toDF()” Method” of the “Delta Table Instance” is used.
"""
dfFromEMPDeltaTable = EMPDeltaTable.toDF()
display(dfFromEMPDeltaTable)

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,null,5000,null,10
7698,BLAKE,MANAGER,7839,null,2850,null,30
7782,CLARK,MANAGER,7839,null,2450,10,10
7566,JONES,MANAGER,7839,null,2975,50,20
7788,SCOTT,ANALYST,7566,null,3000,20,20
7902,FORD,ANALYST,7566,null,3000,50,20
7369,SMITH,CLERK,7902,null,800,500,20
7499,ALLEN,SALESMAN,7698,null,1600,300,30
7521,WARD,SALESMAN,7698,null,1250,500,30
7654,MARTIN,SALESMAN,7698,null,1250,1400,30


In [0]:

"""
Merge Method1 : This syntax is detailed for all columns
EMPDeltaTable.alias("Target")\
    .merge(
            source = df_source.alias("Source"),
            condition = "Target.EMPNO = Source.EMPNO"
          )\
    .whenMatchedUpdate(
                        set =
                        {
                            "Target.ENAME": "Source.ENAME",
                            "Target.JOB": "Source.JOB",
                            "Target.MGR": "Source.MGR",
                            "Target.HIREDATE": "Source.HIREDATE",
                            "Target.SAL": "Source.SAL",
                            "Target.COMM": "Source.COMM",
                            "Target.DEPTNO": "Source.DEPTNO"
                        }
                      )\
    .whenNotMatchedInsert(
                            values =
                            {
                            "Target.ENAME": "Source.ENAME",
                            "Target.JOB": "Source.JOB",
                            "Target.MGR": "Source.MGR",
                            "Target.HIREDATE": "Source.HIREDATE",
                            "Target.SAL": "Source.SAL",
                            "Target.COMM": "Source.COMM",
                            "Target.DEPTNO": "Source.DEPTNO"
                            }
                        )\
    .execute()

"""



# Method 2: the below method is for the syntax using insertall or updateall

EMPDeltaTable.alias("Target")\
    .merge(
            source = df_source.alias("Source"),
            condition = "Target.EMPNO = Source.EMPNO"
          )\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()



   # When the “Merge” Operation is “Performed” using “PySpark”, by default, “Statistics” is “Not Displayed”.

In [0]:
%sql
select * from training.EMP_SCD1

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7839,KING,PRESIDENT,null,null,5000,50,10
7698,BLAKE,MANAGER,7839,null,2850,2000,30
7782,CLARK,MANAGER,7839,null,2450,10,10
7566,JONES,MANAGER,7839,null,2975,50,20
7788,SCOTT,ANALYST,7566,null,3000,20,20
7902,FORD,ANALYST,7566,null,3000,50,20
7369,SMITH,CLERK,7902,null,800,500,20
7499,ALLEN,SALESMAN,7698,null,1600,300,30
7521,WARD,SALESMAN,7698,null,1250,500,30
7654,MARTIN,SALESMAN,7698,null,1250,1400,30
